In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np

In [3]:
import foolbox

In [4]:
from foolbox import zoo

In [5]:
from foolbox import run_sequential
from foolbox import run_parallel

In [6]:
criterion = foolbox.criteria.Misclassification()

### get some example model to attack

In [7]:
model = zoo.get_model('https://github.com/bethgelab/cifar10_challenge')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


imported module: <module 'foolbox_model' from '/gpfs01/bethge/home/jrauber/.foolbox_zoo/871537923d8f2a542aebc37067906f4628c0dc04b0b723f3bf91a53c3e2b1003/foolbox_model.py'>

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Instructions for updating:
Use tf.cast instead.


### get some data and make sure it matches the model's expectations

In [8]:
images, labels = foolbox.utils.samples('cifar10', batchsize=20)

In [9]:
model.channel_axis(), model.bounds()

(3, (0, 255))

In [10]:
images.min(), images.max(), images.dtype, images.shape

(0.0, 255.0, dtype('float32'), (20, 32, 32, 3))

### check the accuracy

In [11]:
accuracy = np.mean(np.argmax(model.forward(images), axis=-1) == labels)
print(f'accuracy on {len(images)} samples: {accuracy}')

accuracy on 20 samples: 0.9


### configure logging to show state of the whole batch en detail

In [12]:
import logging

In [13]:
logging.getLogger().setLevel(logging.INFO)

### instead of calling the attack, we (currently) have to pass a constructor (or similar) to our run functions

In [14]:
# can be function, lambda or simply a class (i.e. its constructor)

# attack_create_fn = foolbox.batch_attacks.GradientAttack
attack_create_fn = foolbox.batch_attacks.L2BasicIterativeAttack
# attack_create_fn = foolbox.batch_attacks.CarliniWagnerL2Attack

## run the attack

In [15]:
attack_create_fn

foolbox.batch_attacks.iterative_projected_gradient.L2BasicIterativeAttack

### run sequentially

In [16]:
advs = run_sequential(attack_create_fn, model, criterion, images, labels)

INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.075
INFO:root:successful for eps = 0.0375
INFO:root:successful for eps = 0.01875
INFO:root:successful for eps = 0.009375
INFO:root:successful for eps = 0.0046875
INFO:root:successful for eps = 0.00234375
INFO:root:successful for eps = 0.001171875
INFO:root:not successful for eps = 0.0005859375
INFO:root:not successful for eps = 0.0008789062499999999
INFO:root:not successful for eps = 0.0010253906249999998
INFO:root:successful for eps = 0.0010986328125
INFO:root:not successful for eps = 0.00106201171875
INFO:root:successful for eps = 0.001080322265625
INFO:root:successful for eps = 0.0010711669921874998
INFO:root:successful for eps = 0.0010665893554687499
INFO:root:not successful for eps = 0.001064300537109375
INFO:root:successful for eps = 0.0010654449462890623
INFO:root:not successful for eps = 0.0010648727416992187
INFO:root:successful for eps = 0.0010651588439941405
INFO:root:1 of

INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.075
INFO:root:successful for eps = 0.0375
INFO:root:successful for eps = 0.01875
INFO:root:successful for eps = 0.009375
INFO:root:successful for eps = 0.0046875
INFO:root:successful for eps = 0.00234375
INFO:root:successful for eps = 0.001171875
INFO:root:successful for eps = 0.0005859375
INFO:root:successful for eps = 0.00029296875
INFO:root:successful for eps = 0.000146484375
INFO:root:successful for eps = 7.32421875e-05
INFO:root:successful for eps = 3.662109375e-05
INFO:root:successful for eps = 1.8310546875e-05
INFO:root:successful for eps = 9.1552734375e-06
INFO:root:successful for eps = 4.57763671875e-06
INFO:root:successful for eps = 2.288818359375e-06
INFO:root:successful for eps = 1.1444091796875e-06
INFO:root:successful for eps = 5.7220458984375e-07
INFO:root:successful for eps = 2.86102294921875e-07
INFO:root:9 of 20 attacks completed
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.15


INFO:root:successful for eps = 0.013374996185302732
INFO:root:16 of 20 attacks completed
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.075
INFO:root:successful for eps = 0.0375
INFO:root:successful for eps = 0.01875
INFO:root:successful for eps = 0.009375
INFO:root:successful for eps = 0.0046875
INFO:root:not successful for eps = 0.00234375
INFO:root:successful for eps = 0.0035156249999999997
INFO:root:successful for eps = 0.0029296875
INFO:root:successful for eps = 0.00263671875
INFO:root:not successful for eps = 0.002490234375
INFO:root:not successful for eps = 0.0025634765625
INFO:root:successful for eps = 0.00260009765625
INFO:root:not successful for eps = 0.002581787109375
INFO:root:successful for eps = 0.0025909423828125
INFO:root:successful for eps = 0.00258636474609375
INFO:root:not successful for eps = 0.002584075927734375
INFO:root:not successful for eps = 0.0025852203369140627
INFO:root:successful for eps = 0.00258579

**took 87s on a V100**

In [17]:
l2s = np.sqrt(np.asarray([adv.distance.value for adv in advs]) * images[0].size)
print(l2s.min(), l2s.max(), np.mean(l2s), np.median(l2s))

2.6939584127705e-06 8.984590165293739 1.1154383108155792 0.48617539076119465


In [18]:
l2s

array([5.90340948e-02, 4.98349804e-01, 2.23367315e-01, 6.71481585e-01,
       4.74000978e-01, 2.51941591e-01, 5.50681080e-01, 2.69395841e-06,
       2.72336066e-06, 8.42169349e-02, 1.75335262e-01, 4.73309270e+00,
       1.75129099e-01, 8.98459017e+00, 1.64437968e+00, 7.39636443e-01,
       1.43317503e-01, 8.92089660e-01, 6.46727624e-01, 1.36138929e+00])

In [19]:
np.array([adv._total_prediction_calls for adv in advs])

array([125, 165, 156, 166, 163, 156, 156,  21,  21, 130, 145, 202, 144,
       209, 184, 168, 134, 172, 168, 181])

In [20]:
np.array([adv._total_gradient_calls for adv in advs])

array([125, 165, 156, 166, 163, 156, 156,  21,  21, 130, 145, 202, 144,
       209, 184, 168, 134, 172, 168, 181])

### run in parallel

In [21]:
advs = run_parallel(attack_create_fn, model, criterion, images, labels)

INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.3
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:root:successful for eps = 0.15
INFO:roo

INFO:root:successful for eps = 0.004101562499999999
INFO:root:not successful for eps = 0.008203124999999999
INFO:root:successful for eps = 0.0099609375
INFO:root:not successful for eps = 0.0010253906249999998
INFO:root:successful for eps = 0.012890625
INFO:root:successful for eps = 0.00322265625
INFO:root:successful for eps = 0.00322265625
INFO:root:not successful for eps = 0.002490234375
INFO:root:not successful for eps = 0.010546875
INFO:root:not successful for eps = 0.004101562499999999
INFO:root:not successful for eps = 0.012890625
INFO:root:successful for eps = 0.0015380859375
INFO:root:successful for eps = 0.20625
INFO:root:not successful for eps = 0.084375
INFO:root:successful for eps = 0.02578125
INFO:root:not successful for eps = 0.015234374999999998
INFO:root:successful for eps = 0.0010986328125
INFO:root:successful for eps = 0.0087890625
INFO:root:successful for eps = 0.030468749999999996
INFO:root:not successful for eps = 0.0087890625
INFO:root:not successful for eps = 0.00

INFO:root:not successful for eps = 0.08587646484375
INFO:root:successful for eps = 0.024563598632812503
INFO:root:not successful for eps = 0.016090393066406247
INFO:root:not successful for eps = 0.008553314208984373
INFO:root:successful for eps = 0.003160285949707031
INFO:root:not successful for eps = 0.029800415039062493
INFO:root:not successful for eps = 0.0025855064392089847
INFO:root:5 of 20 attacks completed
INFO:root:successful for eps = 0.008992767333984375
INFO:root:successful for eps = 0.003163719177246093
INFO:root:not successful for eps = 0.0040294647216796865
INFO:root:successful for eps = 0.004546737670898438
INFO:root:not successful for eps = 0.009936904907226561
INFO:root:successful for eps = 0.012114715576171874
INFO:root:not successful for eps = 0.013373565673828124
INFO:root:not successful for eps = 0.011670684814453125
INFO:root:successful for eps = 0.20540771484375
INFO:root:not successful for eps = 0.08589477539062501
INFO:root:not successful for eps = 0.0245590209

**took 18s on a V100**

In [22]:
l2s = np.sqrt(np.asarray([adv.distance.value for adv in advs]) * images[0].size)
print(l2s.min(), l2s.max(), np.mean(l2s), np.median(l2s))

2.6939584127705e-06 8.249814463212598 1.0778665285606264 0.4862309097541191


In [23]:
l2s

array([5.90340809e-02, 4.98349834e-01, 2.23383190e-01, 6.71418130e-01,
       4.74111986e-01, 2.51989066e-01, 5.50696952e-01, 2.69395841e-06,
       2.72336066e-06, 8.42169126e-02, 1.75335262e-01, 4.71609839e+00,
       1.75144995e-01, 8.24981446e+00, 1.64459345e+00, 7.39598431e-01,
       1.43317489e-01, 8.91994504e-01, 6.46965512e-01, 1.36126251e+00])

In [24]:
np.array([adv._total_prediction_calls for adv in advs])

array([125, 165, 156, 167, 163, 156, 156,  21,  21, 130, 145, 201, 144,
       210, 184, 168, 134, 172, 168, 181])

In [25]:
np.array([adv._total_gradient_calls for adv in advs])

array([125, 165, 156, 167, 163, 156, 156,  21,  21, 130, 145, 201, 144,
       210, 184, 168, 134, 172, 168, 181])

**Note that the speed-up depends a lot on the attack, the model, the batch size and more.**